In [ ]:
%load_ext autoreload
%autoreload 2

# setup

> Setup directories to store data.

In [ ]:
#| default_exp setup

In [ ]:
#| hide
from nbdev.showdoc import *

/home/magyary/anaconda3/envs/kerepesi_2025/lib/python3.12/site-packages/nbdev/doclinks.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources,importlib


# Set data directory

In [ ]:
#| export

from pathlib import Path
import os, json
import requests
from io import BytesIO
import pandas as pd

In [ ]:
#| export

CONFIG_FILE = Path.home() / ".bs-cpg-config.json"
SUBDIR_NAME = ".bs-cpg"

def get_base_data_path():
    """
    Determines the base data path with a clear priority:
    1. BS_CPG_DATA environment variable.
    2. Path stored in ~/.bs-cpg-config.json.
    3. Prompts the user for the path as a last resort.
    """
    # 1. Check for an environment variable (ideal for CI/automation)
    path_str = os.getenv('BS_CPG_DATA')
    
    if path_str:
        base = Path(path_str).expanduser().resolve()
    # 2. Check for the config file (for returning users)
    elif CONFIG_FILE.exists():
        base = Path(json.loads(CONFIG_FILE.read_text())["base_path"]).expanduser().resolve()
    # 3. Prompt user only if no other configuration is found
    else:
        user_input = input("Enter a base path to store data (e.g., ~/data): ")
        base = Path(user_input).expanduser().resolve()
        # Save the configuration so we don't have to ask again
        CONFIG_FILE.write_text(json.dumps({"base_path": str(base)}))
    
    final_path = base / SUBDIR_NAME
    final_path.mkdir(parents=True, exist_ok=True)
    return final_path

In [ ]:
#| export
def read_sample_cpg(columns: list = None, force_download: bool = False):
    """
    Downloads and reads a sample CpG Parquet file.

    This function fetches a sample dataset from the project's GitHub repository.
    It caches the file locally to avoid re-downloading on subsequent calls.

    Args:
        columns (list, optional): A list of columns to read from the file. 
                                  Defaults to None (all columns).
        force_download (bool, optional): If True, forces a re-download of the
                                         file even if it exists locally. 
                                         Defaults to False.

    Returns:
        pd.DataFrame: A DataFrame containing the sample CpG data.
    """
    # Use a permanent link to a specific commit for reproducibility
    sample_url = "https://github.com/magistak/bs-cpg/raw/81f806649a691b766ca658437d857591c2eb6c60/data/sample_cpg_df.parquet" # Example commit hash
    
    # Use your existing function to find the local data directory for caching
    data_dir = get_base_data_path()
    local_file_path = data_dir / "sample_cpg_df.parquet"

    if not local_file_path.exists() or force_download:
        print(f"Downloading sample data to {local_file_path}...")
        response = requests.get(sample_url)
        # Raise an exception for bad status codes (like 404 Not Found)
        response.raise_for_status() 
        local_file_path.write_bytes(response.content)

    return pd.read_parquet(local_file_path, columns=columns)
df = read_sample_cpg(["chromosome", "pos"])
df

,chromosome,pos
1594976,chr11,30607691
41581,chr1,10269003
2926242,chrX,55066102
573570,chr3,169781668
1930113,chr14,59950571
...,...,...
2556779,chr19,17438314
452764,chr2,241331303
1554781,chr10,135171502
898587,chr6,31869119


In [ ]:
#| export

get_base_data_path()

Path('/mnt/idms/home/magyary/.bs-cpg')

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()